In [ ]:
# proof cooperative scheduling on memory fetching is each memory instruction act like an await or yield allow the scheduler to switch the warps
# flood the sm with max blocks with pure compute ie an infinite loop so it can switch to another warp or taken in another warp
# run info cuda sms to see that all 40 sm are working
# run info cuda warps to see all 32 warps per sm
# unable to launch with more than 1024 threads in a block or more than 32 warps per sm
# check max block and grid size

In [ ]:
%%writefile test.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include <unistd.h>

__global__ void infinite_kernel() { while (true) {} }

__global__ void test_kernel() { printf("Hello from test kernel! If you see this, some SMs were still free.\n"); }


int main() {
    printf("Starting test on T4 GPU...\n");

    // T4 has 40 SMs, so launching > 40 blocks ensures full SM occupancy
    // Each block runs an infinite loop, preventing other kernels from running
    // using dim3(x, y, z) type to define the thread in each dimension more explicitly
    // 1024 and 2147483647, 65535, 65535 max limit !!!!!!

    infinite_kernel<<<dim3(40, 1, 1), dim3(1024, 1, 1)>>>();

    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("Error launching infinite kernel: %s\n", cudaGetErrorString(err));
        return -1;
    }

    sleep(1);

    printf("Launching test_kernel...\n");
    test_kernel<<<dim3(1, 1, 1), dim3(1, 1, 1)>>>();

    err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("Error launching test kernel: %s\n", cudaGetErrorString(err));
        return -1;
    }

    printf("Synchronizing device (expected to hang here)...\n");
    err = cudaDeviceSynchronize();  // <--- FREEZES HERE

    if (err != cudaSuccess) { printf("Sync error: %s\n", cudaGetErrorString(err)); }

    printf("I Should not be seen!!!\n");
    return 0;
}


Overwriting test.cu


In [ ]:
!nvcc -arch=sm_75 -o test test.cu
!./test

Starting test on T4 GPU...
Error launching infinite kernel: invalid configuration argument


In [ ]:
# nvidia-smi
# nvidia-smi dmon -s u -c 10